# 0.0 IMPORTS

In [23]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4
import json
import io

In [24]:
# vai ser usado para verificar se as info que eu quero estão no arquivo que eu baixei
queries = [ "machine+learning", "data+science", "kaggle"]
# url que irei usar para biscar esses vídeos
url = "https://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}"
# "search_query" vai receber a palavra que eu quero procurar dentro da variavel "query"
# "&sp" que é um código que vai ordenar os resultados do amis recente para o mais antigo (data de upload)
# "p" você não vai ver esse código no endereço da página do Youtube se for fazer uma consulta direto na plataforma, mas com 
# esse código é possível paginar a busca no Youtube.

In [25]:
# que página vou buscar? eu bisco os vídeos que ordeno pelos amis recentes
# o resultado de busca e a página do vídeo

# precisa pegar a página que retorna os resultados da busca e pegar a pa´gina de cada vídeo que é retornado.
## para fazer isso vamos usar biblioteca chamada "BeautifulSoup" e "requests"
# os datasets usados eu não tenho printis para baixar (como no kaggle). Então vou ter que criar COLETORES para as duas
# páginas no youtube.

# dica de sempre salvar a página que vc pegou com os coletores e está trabalhando no projeto. Motivos:
# - sobrecarregar o servidor se acabar testando muitas cisas várias vezes e acabar sendo bloqueado pelo site do youtube
## soluçõ: baixar todas as pag que precisamos e dados e depois pensar sobre a parte de processamento. o "requests" é uma 
## biblioteca que faz requisições no servidor de hospedagem e ele tem tanto requisições GET como POST. O "BeautifulSoup" será
## usado para fazer o parser.

# 1.0 DATA COLLECTION

In [26]:
# Criando o Coletor da Página de Busca do Youtube.

for query in queries:
    for page in range(1, 21): # para cada query eu quero que ele pegue um determinado número de páginas.
        urll = url.format( query=query, page=page ) # formata a url com sa info que eu quero. Basicamente é como se eu estivesse
        # fazendo manualemente a busca com a palavra que eu quero e salvando a quantidade de páginas que eu desejo.
        print( urll ) # faz um print para verificar como estão sendo geradas essas páginas e se está correto.
        response = rq.get( urll ) # quando se usa a tecnica GET no "requests" ele vai tentar fazer uma requisição no servidor
        # do site usando o endereço que está dentro da variavel "urll"
        
        with open( "./dados_brutos/{}_{}.html".format(query, page), 'w+', encoding="utf-8" ) as output: # abro um arquivo 
            # na dentro da pasta "dados_brutos" e salvo as páginas com os nomes que eu formato usando as informações de "query" 
            # e "page" para diferenciá-los. Vai salvar neste arquivo o código fonte da página que estou tentando pegar.
            output.write( response.text ) # vai transformar a página em dados string (.text) e escrever (write) no arquivo que
            # vai ter o nome já determinado acima.
        time.sleep(2) # para não sobrecarregar o servidor uso o "time.sleep" para dar uns 2 segundos de intervalo entre uma 
        # requisição e outra.

https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=1
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=2
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=3
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=4
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=5
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=6
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=7
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=8
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=9
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=10
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=11
https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=12
https://www.youtube.com/results?search_query=machine+learning

# 2.0 RAW DATA PROCESSING

In [27]:
# Criando o Extrator de Dados da Página de Busca do Youtube.

# Quando executamos o código na última sessão e analisamos as páginas resultantes vemos que as infkrmações importantes para 
# pegarmos destas páginas são a príncipio o título e o link do vídeo.

for query in queries:
    for page in range(1, 21): # vai interar por todas as páginas.
        with open( "./dados_brutos/{}_{}.html".format(query, page), 'r+', encoding="utf-8" ) as inp: 
            page_html = inp.read() # vai ler os arquivos presentes nestas páginas e guardar na variável "page_html"
            
            parsed = bs4.BeautifulSoup( page_html ) # aqui eu uso o BeautifulSoup para entrar na estrutura da página e ajudar
            # a encontrar as informações que nós queremos.
            
            tags = parsed.findAll( "a" ) # vai enontrar todas as tagas com o nome presente dentro dela. Neste caso, como
            # queremos achar as tags links, usamos o "a". No HTML as tags links são representadas por "a".
            
            for e in tags: # vai interar por todos os atributos dentro da variavel "tags" onde estão presenets as tags "a".
                if e.has_attr( "aria-describedby" ): # Dentro do browser, vasculhando o código presente nas páginas do Youtube,
                    # vc chega a conclusão que o atributo que está sendo procurado aqui e que se chama "aria-describedby" está
                    # presente nos links retornados e que são os que eu quero obter.
                    # Nesse primeiro momento vc verifica se a tag "a" tem esse elemento e se sim vc começa a pegar algumas info.
                    link = e[ 'href' ] # pega o endereço do link
                    title = e[ 'title' ] # pega o título do link
                    with open( "parsed_videos.json", 'a+', encoding="utf-8" ) as output: # depois eu vou salvar essas 
                        # infromaçoes em arquivos formato json que são arquivos tipo dicionários e com isso ajuda a dar maior 
                        # flexibilidade na quantidade de info que veem da busca. ficando mais adaptável a possíveis mudanças.
                        data = {"link": link, "title": title, "query": query } # cria o dicionário que será usado no json com 
                        # as info importantes para se ter os vídeos: link, título e qual a query.
                        output.write( "{}\n".format( json.dumps(data) ) ) # vai escrever o arquivo usando o "dump" que
                        # basicamente transforma o dicionário em string json.

# 3.0 RESULT VERIFICATION

In [28]:
parsed.findAll( "a" )

[<a href="https://www.youtube.com/about/" slot="guide-links-primary" style="display: none;">Sobre</a>,
 <a href="https://www.youtube.com/about/press/" slot="guide-links-primary" style="display: none;">Imprensa</a>,
 <a href="https://www.youtube.com/about/copyright/" slot="guide-links-primary" style="display: none;">Direitos autorais</a>,
 <a href="/t/contact_us" slot="guide-links-primary" style="display: none;">Entre em contato</a>,
 <a href="https://www.youtube.com/creators/" slot="guide-links-primary" style="display: none;">Criadores de conteúdo</a>,
 <a href="https://www.youtube.com/ads/" slot="guide-links-primary" style="display: none;">Publicidade</a>,
 <a href="https://developers.google.com/youtube" slot="guide-links-primary" style="display: none;">Desenvolvedores</a>,
 <a href="/t/terms" slot="guide-links-secondary" style="display: none;">Termos</a>,
 <a href="https://www.google.com.br/intl/pt-BR/policies/privacy/" slot="guide-links-secondary" style="display: none;">Privacidade<

# 4.0 COLECT

In [19]:
df = pd.read_json( "parsed_video.json", lines=True )
df.head()

ValueError: Expected object or value

In [ ]:
# Criando o Extrator de Dados da Página de Busca do Youtube.